In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import pandas as pd
import numpy as np
import os
import os.path 
import matplotlib.pyplot as plt
import matplotlib.colors
import copy
import errno
import glob
import time
import calendar
import json
import pickle
import random
import netCDF4
from numpy import savez_compressed
from optparse import OptionParser
from scipy.interpolate import (UnivariateSpline, RectBivariateSpline, RegularGridInterpolator)
from sklearn.metrics import confusion_matrix

In [48]:
csv_files_dir = '/data1/fog/fognn/Hamid/FogNetImport/SHAP/'
folders = sorted(os.listdir(csv_files_dir))
print(folders[30])
df2 = pd.read_csv(os.path.join(csv_files_dir, folders[24], 'VIS_Prob_TEST.csv'), index_col= 0)
print(df2.shape)


G60
(2228, 2)


In [51]:
def calc_mean_prob_df(id1, id2):
    csv_files_dir = '/data1/fog/fognn/Hamid/FogNetImport/SHAP/'
    folders = sorted(os.listdir(csv_files_dir))
    if id1 == 30: 
        df1 = pd.read_csv(os.path.join(csv_files_dir, folders[id1], 'VIS_Prob_TEST.csv'))
        df1 = df1.fillna(0)
    else: 
        df1 = pd.read_csv(os.path.join(csv_files_dir, folders[id1], 'VIS_Prob_TEST.csv'), index_col= 0)
        df1 = df1.fillna(0)
    
    if id2 == 30: 
        df2 = pd.read_csv(os.path.join(csv_files_dir, folders[id2], 'VIS_Prob_TEST.csv'))
        df2 = df2.fillna(0)
    else: 
        df2 = pd.read_csv(os.path.join(csv_files_dir, folders[id2], 'VIS_Prob_TEST.csv'), index_col= 0)
        df2 = df2.fillna(0)


    #print(f"{id1} = {df1.iloc[:, 0].shape} | {id2} = {df2.iloc[:, 0].shape}")
    output = (df1.iloc[:, 0] - df2.iloc[:, 0])
    output = np.mean(output)
    #print(df1.iloc[:, 0], df2.iloc[:, 0])
    #print(f"{id}-{folders[id]}: {df.shape} |{output} ")
    return output


def SHAP_Scores():    
    Weights = [0.2, 0.2, 0.2, 0.2, 0.2, 
               0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 
               0.0333333,  0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 
               0.05, 0.05, 0.05, 0.05, 0.05,
               0.2] 

    G1_idx = [0, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 25, 26, 27, 28, 30]
    G1_weights = [Weights[index] for index in G1_idx]   
    G1_Sub = [calc_mean_prob_df(0, 30), 
                calc_mean_prob_df(5, 1), 
                calc_mean_prob_df(6, 2), 
                calc_mean_prob_df(7, 3), 
                calc_mean_prob_df(8, 4), 
                calc_mean_prob_df(15, 9),  
                calc_mean_prob_df(16, 10), 
                calc_mean_prob_df(17, 11), 
                calc_mean_prob_df(18, 12), 
                calc_mean_prob_df(19, 13), 
                calc_mean_prob_df(20, 14), 
                calc_mean_prob_df(25, 21),
                calc_mean_prob_df(26, 22), 
                calc_mean_prob_df(27, 23), 
                calc_mean_prob_df(28, 24), 
                calc_mean_prob_df(30, 29)]
    G1_score     = sum([a*b for a, b in zip(G1_weights, G1_Sub)])
    G1_score_abs = sum([a*abs(b) for a, b in zip(G1_weights, G1_Sub)])
    
    G2_idx = [1, 5, 9, 10, 11, 15, 16, 17, 21, 22, 23, 25, 26, 27, 29, 30]
    G2_weights = [Weights[index] for index in G2_idx]
    G2_Sub = [calc_mean_prob_df(1, 30), 
              calc_mean_prob_df(5, 0), 
              calc_mean_prob_df(9, 2), 
              calc_mean_prob_df(10, 3), 
              calc_mean_prob_df(11, 4), 
              calc_mean_prob_df(15, 6),  
              calc_mean_prob_df(16, 7), 
              calc_mean_prob_df(17, 8), 
              calc_mean_prob_df(21, 12), 
              calc_mean_prob_df(22, 13), 
              calc_mean_prob_df(23, 14), 
              calc_mean_prob_df(25, 18),
              calc_mean_prob_df(26, 19), 
              calc_mean_prob_df(27, 20), 
              calc_mean_prob_df(29, 24), 
              calc_mean_prob_df(30, 28)]
    G2_score     = sum([a*b for a, b in zip(G2_weights, G2_Sub)])
    G2_score_abs = sum([a*abs(b) for a, b in zip(G2_weights, G2_Sub)])
 
    G3_idx = [2, 6, 9, 12, 13, 15, 18, 19, 21, 22, 24, 25, 26, 28, 29, 30]
    G3_weights = [Weights[index] for index in G3_idx]
    G3_Sub = [calc_mean_prob_df(2, 30), 
              calc_mean_prob_df(6, 0), 
              calc_mean_prob_df(9, 1), 
              calc_mean_prob_df(12, 3), 
              calc_mean_prob_df(13, 4), 
              calc_mean_prob_df(15, 5),  
              calc_mean_prob_df(18, 7), 
              calc_mean_prob_df(19, 8), 
              calc_mean_prob_df(21, 10), 
              calc_mean_prob_df(22, 11), 
              calc_mean_prob_df(24, 14), 
              calc_mean_prob_df(25, 16),
              calc_mean_prob_df(26, 17), 
              calc_mean_prob_df(28, 20), 
              calc_mean_prob_df(29, 23), 
              calc_mean_prob_df(30, 27)]
    G3_score = sum([a*b for a, b in zip(G3_weights, G3_Sub)])
    G3_score_abs = sum([a*abs(b) for a, b in zip(G3_weights, G3_Sub)])
    
    G4_idx = [3, 7, 10, 12, 14, 16, 18, 20, 21, 23, 24, 26, 27, 28, 29, 30]
    G4_weights = [Weights[index] for index in G4_idx]
    G4_Sub = [calc_mean_prob_df(3, 30), 
              calc_mean_prob_df(7, 0), 
              calc_mean_prob_df(10, 1), 
              calc_mean_prob_df(12, 2), 
              calc_mean_prob_df(14, 4), 
              calc_mean_prob_df(16, 5),  
              calc_mean_prob_df(18, 6), 
              calc_mean_prob_df(20, 8), 
              calc_mean_prob_df(21, 9), 
              calc_mean_prob_df(23, 11), 
              calc_mean_prob_df(24, 13), 
              calc_mean_prob_df(25, 15),
              calc_mean_prob_df(27, 17), 
              calc_mean_prob_df(28, 19), 
              calc_mean_prob_df(29, 22), 
              calc_mean_prob_df(30, 26)]

    G4_score = sum([a*b for a, b in zip(G4_weights, G4_Sub)])
    G4_score_abs = sum([a*abs(b) for a, b in zip(G4_weights, G4_Sub)])

    G5_idx     = [4, 8, 11, 13, 14, 17, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30] 
    G5_weights = [Weights[index] for index in G5_idx]
    G5_Sub     = [calc_mean_prob_df(4, 30), 
                calc_mean_prob_df(8, 0), 
                calc_mean_prob_df(11, 1), 
                calc_mean_prob_df(13, 2), 
                calc_mean_prob_df(14, 3), 
                calc_mean_prob_df(17, 5),  
                calc_mean_prob_df(19, 6), 
                calc_mean_prob_df(20, 7), 
                calc_mean_prob_df(22, 9), 
                calc_mean_prob_df(23, 10), 
                calc_mean_prob_df(24, 12), 
                calc_mean_prob_df(26, 15),
                calc_mean_prob_df(27, 16), 
                calc_mean_prob_df(28, 18), 
                calc_mean_prob_df(29, 21), 
                calc_mean_prob_df(30, 25)]

    G5_score = sum([a*b for a, b in zip(G5_weights, G5_Sub)])
    G5_score_abs = sum([a*abs(b) for a, b in zip(G5_weights, G5_Sub)])

    scores = [G1_score,  G2_score, G3_score, G4_score, G5_score]
    scores_abs = [G1_score_abs,  G2_score_abs, G3_score_abs, G4_score_abs, G5_score_abs]
    
    return scores, scores_abs

In [53]:
score, score_abs = SHAP_Scores()
print(f"SHAP Score of G1: {score[0]}")
print(f"SHAP Score of G2: {score[1]}")
print(f"SHAP Score of G3: {score[2]}")
print(f"SHAP Score of G4: {score[3]}")
print(f"SHAP Score of G5: {score[4]}")
print(f"====================================")
print(f"SHAP Abs Score of G1: {score_abs[0]}")
print(f"SHAP Abs Score of G2: {score_abs[1]}")
print(f"SHAP Abs Score of G3: {score_abs[2]}")
print(f"SHAP Abs Score of G4: {score_abs[3]}")
print(f"SHAP Abs Score of G5: {score_abs[4]}")

SHAP Score of G1: 0.0005820333423631726
SHAP Score of G2: -0.00015703046559755775
SHAP Score of G3: 0.0009107430341464334
SHAP Score of G4: 0.0020246283795540016
SHAP Score of G5: -0.003360374788908645
SHAP Abs Score of G1: 0.017983443815464406
SHAP Abs Score of G2: 0.01797191177626995
SHAP Abs Score of G3: 0.01999134425753104
SHAP Abs Score of G4: 0.02032175423559222
SHAP Abs Score of G5: 0.01891770616936736


In [31]:
G1_score, G2_score, G3_score, G4_score, G5_score = SHAP_Scores()
print(f"SHAP Score of G1: {G1_score}")
print(f"SHAP Score of G2: {G2_score}")
print(f"SHAP Score of G3: {G3_score}")
print(f"SHAP Score of G4: {G4_score}")
print(f"SHAP Score of G5: {G5_score}")

SHAP Score of G1: -0.04745178710947809
SHAP Score of G2: -0.045873099200378915
SHAP Score of G3: 0.034714893776016255
SHAP Score of G4: 0.03041422302646109
SHAP Score of G5: 0.028195675482686696
